In [1]:
%%time
import malaya

CPU times: user 12.5 s, sys: 1.77 s, total: 14.3 s
Wall time: 19.5 s


## Deep Siamese network

Purpose of deep siamese network to study semantic similarity between 2 strings, near to 1.0 means more similar. Deep Siamese leverage the power of word-vector, and we also implemented BERT to study semantic similarity and BERT leverage the power of attention!

## List deep siamese models

In [2]:
malaya.similarity.available_deep_siamese()

['self-attention', 'bahdanau', 'dilated-cnn']

* ``'self-attention'`` - Fast-text architecture, embedded and logits layers only with self attention.
* ``'bahdanau'`` - LSTM with bahdanau attention architecture.
* ``'dilated-cnn'`` - Pyramid Dilated CNN architecture.

## Load deep siamese models

In [2]:
string1 = 'Pemuda mogok lapar desak kerajaan prihatin isu iklim'
string2 = 'Perbincangan isu pembalakan perlu babit kerajaan negeri'
string3 = 'kerajaan perlu kisah isu iklim, pemuda mogok lapar'
string4 = 'Kerajaan dicadang tubuh jawatankuasa khas tangani isu alam sekitar'

#### Load bahdanau model

In [5]:
model = malaya.similarity.deep_siamese('bahdanau')

#### Calculate similarity between 2 strings

`predict` need to give 2 strings, left and right string

In [11]:
model.predict(string1, string2)

0.4267301

In [13]:
model.predict(string1, string3)

0.28711933

#### Calculate similarity more than 2 strings

`predict_batch` need to give 2 lists of strings, left and right strings

In [17]:
model.predict_batch([string1, string2], [string3, string4])

array([0.39504164, 0.33375728], dtype=float32)

#### Load self-attention model

In [19]:
model = malaya.similarity.deep_siamese('self-attention')

In [24]:
model.predict_batch([string1, string2], [string3, string4])

array([0.08130383, 0.09907728], dtype=float32)

#### Load dilated-cnn model

In [26]:
model = malaya.similarity.deep_siamese('dilated-cnn')

In [27]:
model.predict_batch([string1, string2], [string3, string4])

array([0.1886251 , 0.00937402], dtype=float32)

## Calculate similarity using doc2vec

We need to load word vector provided by Malaya.

Important parameters,
1. `aggregation`, aggregation function to accumulate word vectors. Default is `mean`.

    * ``'mean'`` - mean.
    * ``'min'`` - min.
    * ``'max'`` - max.
    * ``'sum'`` - sum.
    * ``'sqrt'`` - square root.
   
2. `similarity` distance function to calculate similarity. Default is `cosine`.

    * ``'cosine'`` - cosine similarity.
    * ``'euclidean'`` - euclidean similarity.
    * ``'manhattan'`` - manhattan similarity.

#### Using word2vec

I will use `load_news`, word2vec from wikipedia took a very long time. wikipedia much more accurate.

In [3]:
embedded_news = malaya.word2vec.load_news(64)
w2v_wiki = malaya.word2vec.word2vec(embedded_news['nce_weights'],
                                    embedded_news['dictionary'])

In [4]:
malaya.similarity.doc2vec(w2v_wiki, string1, string2)

0.9181415736675262

In [5]:
malaya.similarity.doc2vec(w2v_wiki, string1, string4)

0.9550771713256836

In [7]:
malaya.similarity.doc2vec(w2v_wiki, string1, string4, similarity = 'euclidean')

0.4642694249990522

Different similarity function different percentage.

**So you can try use fast-text and elmo to do the similarity study.**

## Calculate similarity using summarizer

We can use extractive summarization model `malaya.summarize.deep_extractive()` to get strings embedded and calculate similarity between the vectors.

In [4]:
deep_summary = malaya.summarize.deep_extractive(model = 'skip-thought')

In [5]:
malaya.similarity.summarizer(deep_summary, string1, string3)

0.8722701370716095

## BERT model

BERT is the best similarity model in term of accuracy, you can check similarity accuracy here, https://malaya.readthedocs.io/en/latest/Accuracy.html#similarity. But warning, the model size is 700MB! Make sure you have enough resources to use BERT, and installed bert-tensorflow first,

In [3]:
model = malaya.similarity.bert()

In [4]:
model.predict(string1, string3)

0.97767043

In [5]:
model.predict_batch([string1, string2], [string3, string4])

array([0.9253927, 0.0317315], dtype=float32)

**BERT is the best!**

## Topics similarity

If you are interested in multiple topics searching inside a string when giving set of topics to supervised, Malaya provided some interface and topics related to political landscape in Malaysia

In [2]:
news = 'najib razak dan mahathir mengalami masalah air di kemamam terengganu'
second_news = 'ikat penyedia perkhidmatan jalur lebar Telekom Malaysia (TM) perlu mencari jalan penyelesaian bagi meningkatkan akses capaian Internet ke seluruh negara, kata Menteri Komunikasi dan Multimedia, Gobind Singh Deo. Beliau berkata menjadi dasar kerajaan untuk membekalkan akses Internet jalur lebar kepada semua dan memberi penekanan kepada kualiti perkhidmatan yang terbaik. "Dasar kerajaan untuk bekalkan akses kepada semua bukan sekadar pembekalan sahaja tetapi beri penekanan kepada kualiti perkhidmatan yang baik dan dapat bersaing dengan negara lain pada tahap antarabangsa," kata Gobind Singh menerusi catatan di laman rasmi Twitter beliau, malam tadi. Beliau berkata demikian sebagai respons terhadap aduan beberapa pengguna Twitter berhubung akses Internet yang masih tidak stabil serta harga yang tidak berpatutan di beberapa lokasi di seluruh negara.'

## Topics provided by malaya

#### Topics

In [3]:
malaya.topic.topic['sosial']

['sosial', 'kehidupan', 'taraf hidup', 'sosiologi', 'keusahawan', 'masyarakat']

#### Influencer

In [4]:
malaya.topic.influencer['mahathir']

['tun mahathir',
 'madey',
 'dr mahathir',
 'tun m',
 'mahathir',
 'madir',
 'dr m',
 'mahathir muhamad']

#### location

In [5]:
malaya.topic.location[0]

{'negeri': 'JOHOR', 'parlimen': 'SEGAMAT', 'dun': 'BULOH KASAP'}

#### wakil rakyat

In [6]:
malaya.topic.calon[0]

{'KodN': 1,
 'KodParlimen': 1,
 'KodKawasan': 1,
 'JenisKawasan': 'PARLIMEN',
 'susunan': 2,
 'NamaCalon': 'DATUK ZAHIDI BIN ZAINUL ABIDIN',
 'parti': 'BN'}

## Train fuzzy text similarity

I want to train topics related when given a string. You can give any corpus, the format is,

```python
{'left':['right1','right2']}
```

In [7]:
fuzzy = malaya.similarity.fuzzy(malaya.topic.topic)

In [8]:
fuzzy.get_similarity(news,fuzzy_ratio = 60)

['tan sri mokhzani mahathir', 'masalah air', 'mahathir', 'najib razak']

In [9]:
fuzzy.get_similarity(second_news,fuzzy_ratio = 90)

['pendidikan',
 'sosial media',
 'politik',
 'kerajaan',
 'telekom malaysia',
 'twitter',
 'teknologi',
 'internet']

## Train bag-of-word text similarity

I want to train topics related when given a string. You can give any corpus, the format is,

```python
{'left':['right1','right2']}
```

bag-of-word text similarity fitted by using character wised n-gram.

`vectorizer` supported `['tfidf','count','skip-gram']`.

In [10]:
tfidf = malaya.similarity.bow(malaya.topic.topic,vectorizer = 'tfidf')

In [11]:
tfidf.get_similarity(second_news)

['perkhidmatan awam', 'kkmm', 'universiti islam antarabangsa', 'twitter']

In [12]:
count = malaya.similarity.bow(malaya.topic.topic,vectorizer = 'count')

In [13]:
count.get_similarity(second_news)

['timbalan perdana menteri',
 'parti islam semalaysia',
 'pendidikan',
 '1malaysia',
 'gaji menteri',
 'mic',
 'bebas tahanan',
 'twitter',
 'infrastruktur',
 'suruhanjaya pilihan raya malaysia',
 'perkasa',
 'pakatan harapan',
 'kerajaan',
 'datuk seri ti lian ker',
 'tentera malaysia',
 'gerakan',
 'universiti islam antarabangsa',
 'ptptn',
 'rela',
 'ahli dewan undangan negeri',
 'teknologi',
 'politik',
 'telekom malaysia',
 'kkmm',
 'kementerian dalam negeri',
 'perkhidmatan awam',
 'bursa malaysia',
 'parti pribumi bersatu malaysia',
 'ppbm',
 'hutang negara',
 'menyiasat skandal',
 'majlis pakatan harapan',
 'perdana menteri',
 'menteri pertahanan']

In [14]:
skip = malaya.similarity.bow(malaya.topic.topic,vectorizer = 'skip-gram')

In [15]:
skip.get_similarity(second_news)

[]

## Train siamese network text similarity

All parameters supported,
```python
    """
    Train a deep siamese network for text similarity

    Parameters
    ----------
    dictionary: dict
        format {'left':['right']}
    epoch: int, (default=5)
        iteration numbers
    batch_size: int, (default=32)
        batch size for every feed, batch size must <= size of corpus
    embedding_size: int, (default=256)
        vector size representation for a word
    output_size: int, (default=100)
        encoder output size, bigger means more vector definition
    maxlen: int, (default=100)
        max length of a string to be train
    ngram: tuple, (default=(1,4))
        n-grams size to train a corpus
    num_layers: int, (default=100)
        number of bidirectional rnn layers

    Returns
    -------
    _DEEP_SIAMESE_SIMILARITY: malaya.similarity._DEEP_SIAMESE_SIMILARITY class
    """
```

In [16]:
siamese = malaya.similarity.deep_siamese(malaya.topic.topic,epoch=3)
siamese.get_similarity(news)

minibatch loop: 100%|██████████| 137/137 [01:40<00:00,  1.54it/s, accuracy=1, cost=0.0514]    


['parti islam semalaysia',
 'pusat transformasi bandar',
 'malaysia baru',
 'mic',
 'bridge city park',
 'suruhanjaya pilihan raya malaysia',
 'kotak undi',
 'lgbt',
 'tentera malaysia',
 'dewan rakyat',
 'isu kemiskinan',
 'undi rosak',
 'produk berbahaya',
 'politik',
 'telekom malaysia',
 'bank negara',
 'kertas undi',
 'malay mail',
 'gaji minimum',
 'donald trump',
 'najib razak',
 'bank malaysia',
 'humanoid',
 'perkhidmatan awam',
 'rosmah mansur',
 'isu dadah',
 'stock market malaysia',
 'bursa malaysia',
 'pusat daerah mangundi',
 'undi pos',
 'universiti teknologi malaysia',
 'hutang negara',
 'makro-ekonomi',
 'rtm',
 'pengangkutan awam']

You can speed up your training iteration by using [malaya-gpu](https://pypi.org/project/malaya-gpu/)

After you trained, actually you save that model by using method `save_model`. Just provide directory you want to save.

In [17]:
siamese.save_model('siamese')

In [18]:
!ls siamese

checkpoint                     model.ckpt.meta
model.ckpt.data-00000-of-00001 model.json
model.ckpt.index


You can load your model but need to use interface provided by malaya, `malaya.similarity.load_siamese`

In [19]:
siamese = malaya.similarity.load_siamese('siamese')

INFO:tensorflow:Restoring parameters from siamese/model.ckpt


In [20]:
siamese.get_similarity(news)

['pilihan raya umum ke-14',
 'parti islam semalaysia',
 'malaysia baru',
 'pengedar dadah',
 'suruhanjaya pilihan raya malaysia',
 'kotak undi',
 'lgbt',
 'makanan',
 'tentera malaysia',
 'gerakan',
 'isu kemiskinan',
 'undi rosak',
 'produk berbahaya',
 'bloomberg',
 'telekom malaysia',
 'bank negara',
 'kertas undi',
 'malay mail',
 'gaji minimum',
 '1mdb',
 'najib razak',
 'bank malaysia',
 'humanoid',
 'perkhidmatan awam',
 'rosmah mansur',
 'isu dadah',
 'stock market malaysia',
 'bursa malaysia',
 'undi pos',
 'universiti teknologi malaysia',
 'hutang negara',
 'makro-ekonomi',
 'rtm',
 'pengangkutan awam']

## Train skipthought text similarity

All parameters supported,

```python
    """
    Train a deep skip-thought network for text similarity

    Parameters
    ----------
    dictionary: dict
        format {'left':['right']}
    epoch: int, (default=5)
        iteration numbers
    batch_size: int, (default=32)
        batch size for every feed, batch size must <= size of corpus
    embedding_size: int, (default=256)
        vector size representation for a word
    maxlen: int, (default=100)
        max length of a string to be train
    ngram: tuple, (default=(1,4))
        n-grams size to train a corpus

    Returns
    -------
    _DEEP_SIMILARITY: malaya.similarity._DEEP_SIMILARITY class
    """
```

In [21]:
skipthought = malaya.similarity.deep_skipthought(malaya.topic.topic,epoch=3)
skipthought.get_similarity(news)

minibatch loop: 100%|██████████| 137/137 [01:17<00:00,  1.90it/s, cost=0.342] 


['pilihan raya umum ke-14',
 'pusat transformasi bandar',
 'hari raya',
 'nga kor ming',
 'programming language',
 '#fakenews',
 'mikro-ekonomi',
 'datuk seri azmin ali',
 'recep tayyip erdogan',
 'k-pop',
 'malaysia-indonesia',
 'tengku razaleigh hamzah',
 'anthony loke siew fook',
 'lee kuan yew',
 'rais yatim',
 'undi rosak',
 'kkmm',
 'inisiatif peduli rakyat',
 'tunku ismail idris',
 'pusat daerah mangundi',
 'makro-ekonomi',
 'new straits times']

You can speed up your training iteration by using [malaya-gpu](https://pypi.org/project/malaya-gpu/)

After you trained, actually you save that model by using method `save_model`. Just provide directory you want to save.

In [22]:
skipthought.save_model('skipthought')

In [23]:
!ls skipthought

checkpoint                     model.ckpt.meta
model.ckpt.data-00000-of-00001 model.json
model.ckpt.index


You can load your model but need to use interface provided by malaya, `malaya.similarity.load_skipthought`

In [24]:
skipthought = malaya.similarity.load_skipthought('skipthought')

INFO:tensorflow:Restoring parameters from skipthought/model.ckpt


In [25]:
skipthought.get_similarity(news)

['pilihan raya umum ke-14',
 'pusat transformasi bandar',
 'hari raya',
 'nga kor ming',
 'programming language',
 '#fakenews',
 'mikro-ekonomi',
 'datuk seri azmin ali',
 'recep tayyip erdogan',
 'k-pop',
 'malaysia-indonesia',
 'tengku razaleigh hamzah',
 'anthony loke siew fook',
 'lee kuan yew',
 'rais yatim',
 'undi rosak',
 'kkmm',
 'inisiatif peduli rakyat',
 'tunku ismail idris',
 'pusat daerah mangundi',
 'makro-ekonomi',
 'new straits times']

## Using fuzzy for location

In [26]:
malaya.similarity.fuzzy_location('saya suka makan sate di sungai petani')

{'negeri': [], 'parlimen': ['sungai petani'], 'dun': []}

## Check location from a string

In [27]:
malaya.similarity.is_location('sungai petani')

True